# Task: discover user communities in the user-user network

in task 2 you produced the list of edges.

now you will need to transform those into two additional data structures:
* a set of graph nodes  
* a set of adjacency lists
 
 To understand their formats, consider this small example:
edges RDD:  `([('u1','u2','w1'),('u2','u3','w2'),('u2','u4','w3'),('u1','u5','w4')])`

* the nodes set contains the unique userIds that are found in the edges RDD:
  `['u3', 'u4', 'u2', 'u5', 'u1']`
* the adjacency lists are in the form of a python dictionary of dictionaries, as in this example:
```
[('u3', {'u2': 'w2'}),
 ('u2', {'u3': 'w2', 'u1': 'w1', 'u4': 'w3'}),
 ('u4', {'u2': 'w3'}),
 ('u5', {'u1': 'w4'}),
 ('u1', {'u5': 'w4', 'u2': 'w1'})]
 ```

once you have produced these datsa structures, you add them to a `Graph` object. This Graph is then passed on to a library that computes Single-source-Shortest-Paths (SSSP):
`graph = Graph('user-user-network', allNodes.collect(), allAdjLists)`

## available methods
The following library methods are provided to you to implement the Girwan-Newman algorithm, using this Graph as input:

### SSSP

which is available in module
`from comscan.algorithms.shortest_path.single_source_shortest_paths_dijkstra`

and is defined as:

```
def single_source_shortest_paths_dijkstra(graph, source, cutoff=None):
    """
    Provides Single Source Shortest Path implementation for a weighted graph
    :param graph: Weighted graph implemented with adjacency list.
    Edges in the graph are in the form of (u,v,w) where u,v are long and w is float
    :param source: source node
    :param cutoff: threshold value of largest possible distance in shortest paths.
    :return: all possible shortest paths from source to every other node in the graph
    """
```
this will return the set of all shortest paths in the graph, starting from a **specific source node**, e.g.

`paths, distances = single_source_shortest_paths_dijkstra(graph, source=<source node>)`

### computing the betweeness of each edge in the graph, given the shortest paths:

`def compute_edge_betweenness(shortest_paths_rdd: RDD) -> RDD:
    """Compute edge betweenness centrality of a graph in a distributed fashion given the shortest paths `
 
Using these methods and your knowledge of the GW algorithm, you should try and implement a Spark version of it where RDDs are parallelised where possible so that as much of the computation as possible occurs in parallel.

### removing edges from the graph:
`graph.remove_edge(u,v)`  where `(u,v)` is an edge (works by side-effect. no need to reassign the graph variable)

### calculating the number of connected components in the graph, once edges are removed (removing edges may produce multiple CC)
`ncc = number_connected_components(graph)`

### other utility functions
`get_top_k_edges_with_highest_betweenness(sc, graph, k)` 

will give you the top-k edges with highest betweeness, as you will look for connected components (communities) by repeatedly removing high-betweeness edges from the original graph

`_remove_self_loops(graph)` updates `graph` by removing all loops from a node to itself

please note:
* you may ask workers to call the shortst-path method. If you do so, be careful as the method requires access to the entire graph. So you must **broadcast** the graph data structure to the workers prior to letting them work in it.
* the total betweenees of an edge can be computed *piecemeal* because it can be obtained by adding up partial betweeness values. 

You will produce a set of communities. Each of them is represented by a set of node IDs, so you end up with a list of lists:

```[ [u,v,...], [u2, v2, ...] ]```

save the result to file as indicated below. don't forget to use your `<id>` to make your file name unique.

In [2]:
import pandas as pd
import pyspark.sql.functions as f
import os,sys

In [3]:
# IN:
EDGES_SMALL_PARQUET = "/FileStore/tables/edges-small.parquet"
# EDGES_100K_PARQUET = "/FileStore/tables/edges-100K.parquet"

# ALL_NODES_100K_TEXT = "/FileStore/tables/nodes-100K.txt"
# ALL_ADJLIST_100K_TEXT =  "/FileStore/tables/adjlist-100K.txt"

# Checkpoint files (to facilitate partial rerun during debugging)
ALL_NODES_SMALL_TEXT = "/FileStore/tables/180571930/nodes-small.txt"
ALL_ADJLIST_SMALL_TEXT =  "/FileStore/tables/180571930/adjlist-small.txt"
SHORTEST_PATH_SMALL =   "/FileStore/tables/180571930/shortest-paths-small.txt"

# SHORTEST_PATH_50K =   "/FileStore/tables/shortest-paths-50k.txt"
# SHORTEST_PATH_100K =   "/FileStore/tables/shortest-paths-100k.txt"

# OUT . the output you should produce containing all discovered communities (list of lists of graph nodes)
# CC_SMALL = "/FileStore/tables/ConnectedComponents-small.txt"
CC_SMALL = "/FileStore/tables/180571930/ConnectedComponents-small.txt"

In [4]:
# Import modules
from comscan.model.graph import Graph
from comscan.algorithms.connected import connected_components
from comscan.algorithms.connected import number_connected_components
from comscan.distributed_algorithms.betweenness import compute_edge_betweenness
from comscan.distributed_algorithms.shortest_paths import compute_shortest_paths

In [5]:
def _remove_self_loops(graph):
    for n, neighbors in graph.adj_list.items():
        if n in neighbors:
            graph.remove_edge(n, n)

In [6]:
def get_top_k_edges_with_highest_betweenness(sc, graph, k):
  shortest_paths_rdd = compute_shortest_paths(sc, graph)
  edge_betweenness_rdd = compute_edge_betweenness(shortest_paths_rdd)
  top_k_betweenness = edge_betweenness_rdd.sortBy(lambda x: x[1], ascending=False).take(k)
  # print("top_k_betweenness" + str(top_k_betweenness.collect()))
  # print("top_k_betweenness" + str(top_k_betweenness.take(100)))
  top_k_edges = list(map(lambda x: x[0],top_k_betweenness))
  return top_k_edges

# your code here.
once you have created:
* nodes
* adjLists 
as explained above, you *may* save them here for checkpointing.
however please note that there is no overwrite mode for pickle files. please ask a demonstrator on how to delete files from Databricks' DBFS

In [8]:
# 1. Create a Graph
# 1.1 Edge: Convert dataframe to rdd
edges = spark.read.parquet(EDGES_SMALL_PARQUET)  # DataFrame
edgesRdd = edges.rdd                             
# edgesRdd.collect()

In [9]:
edges.printSchema()

root
-- userId1: integer (nullable = true)
-- userId2: integer (nullable = true)
-- weight: double (nullable = true)

In [10]:
edges.agg(f.countDistinct('userId1')).show() # 609
edges.agg(f.countDistinct('userId2')).show() # 609

+-----------------------+
count(DISTINCT userId1)|
+-----------------------+
 609|
+-----------------------+

+-----------------------+
count(DISTINCT userId2)|
+-----------------------+
 609|
+-----------------------+

In [11]:
# 1.2 Create Node
nodesRdd = edgesRdd.keys().union(edgesRdd.values()).distinct()  
nodesRdd.count() # 610

Out[ 10 ]: 610

In [12]:
# 1.3 Create Adjacency List - [(x[0], {x[1]: x[2]}), ..]
edges_Rdd = edgesRdd.map(lambda row : (row[0], {row[1]:row[2]}))             # [(1, {76: 0.0344}),...]
edges_RRdd = edgesRdd.map(lambda row : (row[1], {row[0]:row[2]}))            # [(76, {1: 0.0344}),...]
adjLists = edges_Rdd.union(edges_RRdd).reduceByKey(lambda x, y: {**x, **y})  # [(1, {76: 0.0344}),(1, {510: 0.0384}),...()]
# adjLists.count() # 610
adjLists.take(10)

Out[ 11 ]: 
[(536,
 {1: 0.1,
 3: 1.0,
 4: 0.16666666666666666,
 5: 0.07692307692307693,
 6: 0.029411764705882353,
 7: 0.14285714285714285,
 8: 0.07142857142857142,
 9: 1.0,
 10: 1.0,
 11: 0.08333333333333333,
 12: 0.5,
 14: 0.1111111111111111,
 15: 0.3333333333333333,
 16: 0.3333333333333333,
 17: 0.1111111111111111,
 18: 0.07692307692307693,
 19: 0.058823529411764705,
 21: 0.14285714285714285,
 22: 0.5,
 23: 0.25,
 24: 0.2,
 25: 1.0,
 26: 0.09090909090909091,
 27: 0.3333333333333333,
 28: 0.08333333333333333,
 29: 0.3333333333333333,
 30: 0.5,
 31: 1.0,
 32: 0.2,
 33: 0.09090909090909091,
 34: 0.25,
 35: 0.2,
 36: 1.0,
 37: 0.1,
 38: 0.05263157894736842,
 39: 0.25,
 40: 0.06666666666666667,
 41: 0.3333333333333333,
 42: 0.058823529411764705,
 43: 0.045454545454545456,
 45: 0.0625,
 46: 0.07142857142857142,
 47: 0.2,
 49: 0.3333333333333333,
 50: 0.3333333333333333,
 51: 0.3333333333333333,
 52: 0.5,
 54: 0.08333333333333333,
 56: 0.05555555555555555,
 57: 0.05555555555555555,
 58: 0.038461538461538464,
 59: 0.16666666666666666,
 60: 1.0,
 61: 1.0,
 62: 0.25,
 63: 0.1,
 64: 0.07692307692307693,
 65: 1.0,
 66: 0.09090909090909091,
 67: 0.5,
 68: 0.045454545454545456,
 69: 0.3333333333333333,
 70: 0.3333333333333333,
 71: 0.5,
 72: 0.1111111111111111,
 73: 1.0,
 75: 0.5,
 76: 0.25,
 78: 0.125,
 79: 0.3333333333333333,
 80: 0.3333333333333333,
 81: 0.1111111111111111,
 82: 0.08333333333333333,
 83: 0.3333333333333333,
 84: 0.041666666666666664,
 85: 1.0,
 86: 0.16666666666666666,
 88: 1.0,
 89: 0.5,
 91: 0.041666666666666664,
 93: 0.08333333333333333,
 94: 0.058823529411764705,
 95: 0.5,
 96: 0.125,
 97: 1.0,
 98: 0.5,
 99: 0.08333333333333333,
 100: 0.16666666666666666,
 101: 0.5,
 102: 0.058823529411764705,
 103: 0.14285714285714285,
 104: 0.16666666666666666,
 105: 0.16666666666666666,
 107: 0.1111111111111111,
 108: 1.0,
 109: 0.04,
 110: 0.3333333333333333,
 111: 0.25,
 112: 0.08333333333333333,
 113: 0.3333333333333333,
 114: 1.0,
 115: 0.1,
 116: 0.09090909090909091,
 117: 0.038461538461538464,
 118: 0.5,
 119: 0.3333333333333333,
 121: 0.06666666666666667,
 122: 0.125,
 123: 1.0,
 124: 0.25,
 125: 0.25,
 126: 0.05555555555555555,
 127: 1.0,
 128: 0.5,
 129: 0.25,
 130: 0.08333333333333333,
 131: 0.3333333333333333,
 132: 0.125,
 133: 0.07142857142857142,
 134: 0.1111111111111111,
 135: 0.08333333333333333,
 136: 0.047619047619047616,
 137: 0.1111111111111111,
 140: 0.058823529411764705,
 141: 0.2,
 142: 0.07142857142857142,
 143: 0.5,
 144: 0.07692307692307693,
 145: 0.125,
 147: 0.5,
 148: 1.0,
 149: 0.25,
 152: 0.5,
 153: 0.3333333333333333,
 154: 1.0,
 155: 1.0,
 156: 0.09090909090909091,
 157: 1.0,
 159: 0.2,
 160: 0.0625,
 162: 0.09090909090909091,
 164: 0.25,
 165: 0.25,
 166: 0.14285714285714285,
 167: 0.16666666666666666,
 168: 0.5,
 169: 0.16666666666666666,
 170: 0.07142857142857142,
 171: 0.3333333333333333,
 172: 0.3333333333333333,
 173: 0.09090909090909091,
 174: 0.047619047619047616,
 176: 0.06666666666666667,
 177: 0.1111111111111111,
 178: 0.09090909090909091,
 179: 0.05263157894736842,
 180: 1.0,
 181: 0.045454545454545456,
 182: 0.06666666666666667,
 183: 0.2,
 186: 0.14285714285714285,
 187: 0.125,
 188: 1.0,
 189: 0.3333333333333333,
 190: 1.0,
 191: 0.16666666666666666,
 192: 0.09090909090909091,
 193: 0.25,
 194: 1.0,
 195: 0.125,
 196: 0.3333333333333333,
 197: 0.3333333333333333,
 198: 0.125,
 199: 0.1111111111111111,
 200: 0.1111111111111111,
 201: 0.14285714285714285,
 202: 0.07692307692307693,
 203: 0.25,
 204: 0.5,
 205: 1.0,
 208: 0.3333333333333333,
 209: 0.5,
 210: 1.0,
 211: 1.0,
 212: 0.25,
 213: 1.0,
 215: 0.3333333333333333,
 217: 0.05,
 219: 0.08333333333333333,
 220: 0.14285714285714285,
 221: 0.14285714285714285,
 222: 0.16666666666666666,
 223: 0.2,
 224: 0.3333333333333333,
 226: 0.125,
 227: 0.14285714285714285,
 229: 0.047619047619047616,
 230: 0.14285714285714285,
 231: 0.3333333333333333,
 232: 0.125,
 233: 0.2,
 234: 0.3333333333333333,
 235: 0.05,
 237: 1.0,
 238: 1.0,
 239: 

### checkpoint save nodes and lists

In [14]:
%fs
rm -r "/FileStore/tables/180571930/nodes-small.txt"

res0: Boolean = false

In [15]:
%fs
rm -r "/FileStore/tables/180571930/adjlist-small.txt"

res1: Boolean = true

In [16]:
# note this will fail if the files already exist
nodesRdd.saveAsPickleFile(ALL_NODES_SMALL_TEXT)
adjLists.saveAsPickleFile(ALL_ADJLIST_SMALL_TEXT) 

### checkpoint. load nodes and adjlist from file

In [18]:
## load up all nodes from file, as a RDD. However the Graph object expects to see a list, thus we use collect() here
nodes_rdd = sc.pickleFile(ALL_NODES_SMALL_TEXT)
nodes = nodes_rdd.collect()

## adjLists is an RDD representing a dictionary of dictionaries (the adjlist). the collectAsMap() method reconstructs the original dictionary
adjLists = sc.pickleFile(ALL_ADJLIST_SMALL_TEXT).collectAsMap()

In [19]:
type(adjLists)

Out[ 14 ]: dict

create a `Graph()`   object and add all nodes and adjacency lists

In [21]:
# 1.4 Create a graph
graph = Graph('user-user-network', nodes, adjLists)
# type(graph) # comscan.model.graph.Graph

In [22]:
# graph.nodes          # [200, 400, 600, 401, 1, 601, 201, 2, 602, 202]
# graph.edges          # <generator object edges at 0x7f40627c4e60>

# your Girwan-Newman implementation goes here

In [24]:
# 2. Girwan-Newman algorithm
# 2.1 Broadcast the graph data structure to the workers prior to letting them work in it.
sc.broadcast(nodes)
sc.broadcast(adjLists)

In [25]:
# 2.2 Removing all loops from a node to itself
_remove_self_loops(graph)

In [26]:
k = 10000        # Set k
targetNCC = 15   # Set a target ncc
count = 0        # the number of cut edges
ncc = number_connected_components(graph)
cc = list(connected_components(graph))

while ncc < targetNCC:
  # 2.3 Get top-k edges with highest betweeness
  top_k_edges = get_top_k_edges_with_highest_betweenness(sc, graph, k)
    
  # 2.4 Removing edges from the graph:
  for edge in top_k_edges:
    graph.remove_edge(edge[0], edge[1])
  count += k
  
  # 2.5 Calculate the number of connected components
  newNcc = number_connected_components(graph)
  
  # 2.6 Calculate connected components
  newCc = list(connected_components(graph))
  
  # 2.7 Community Detect
  if(newNcc != ncc)
    ncc = newNcc
    cc = newCc
    print("Detect a new community, Remove %i edges" % count)

In [27]:
# 3 Save file 
# 3.1 list->rdd
ccSmallRdd = sc.parallelize(cc)
# ccSmallRdd.collect()

In [28]:
%fs
rm -r "/FileStore/tables/180571930/ConnectedComponents-small.txt"

In [29]:
ccSmallRdd.saveAsPickleFile(CC_SMALL)

In [30]:
# 3.2 Test 
test = sc.pickleFile(CC_SMALL)
test.collect()